In [1]:
%matplotlib inline
from __future__ import division
import sys
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import sklearn.preprocessing as preprocessing
import sklearn.feature_extraction as feature_extraction
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix, make_scorer, recall_score, classification_report, roc_curve, auc,roc_auc_score
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV, ParameterSampler
from sklearn.kernel_approximation import RBFSampler
from scipy.stats.distributions import expon
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import time


matplotlib.rcParams['figure.figsize'] = (13.0, 13.0)
#Funciones
def getMeanROC(r):
    fpr = 0
    tpr = 0
    _ = 0
    for i in r:
        if not isinstance(fpr, list) or not isinstance(tpr, list) or not isinstance(_, list):
            fpr = [i[0]]
            tpr = [i[1]]
            _   = [i[2]]
        else:
            fpr = np.append(fpr, [i[0]], axis=0)
            tpr = np.append(tpr, [i[1]], axis=0)
            _   = np.append(_, [i[2]], axis=0)
    return [np.mean(fpr, axis=0),np.mean(tpr, axis=0),np.mean(_, axis=0)]
def confidence_level_roc(y_test,y_pred):
    #print("Original ROC area: {:0.3f}".format(roc_auc_score(y_test, y_pred)))
    n_bootstraps = 1000
    #rng_seed = rs  # control reproducibility
    bootstrapped_scores = []

    #rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = np.random.random_integers(0, len(y_test) - 1, len(y_pred))
        if len(np.unique(y_test[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue

        score = roc_auc_score(y_test[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
    #print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(confidence_lower, confidence_upper))
    return (confidence_lower, confidence_upper)
def fullClassificationReport(estimator, X, y, y_test, y_pred, cv=2, labelNames =['0','1'], estimatorName=None):
    m = confusion_matrix(y_test, y_pred)
    print("----------------------")
    print("----------------------")
    if estimatorName:
        print("Modelo : " + estimatorName)     
    else:
        print("Modelo : " + estimator.loss)
    print("----------------------")
    print("----------------------")
    print("Matriz de Confusion")
    print("----------------------")
    print(m)
    print("----------------------")
    print("Reporte de Clasificacion")
    print("----------------------")
    print(classification_report(y_test, y_pred, target_names=t_names))
    print("----------------------")
    print("Validacion Cruzada k=%s" % cv)
    print("----------------------")
    print("Clase 1 : Desercion\nClasificaciones Correctas")
    print("----------------------")
    crossv = cross_validation.cross_val_score(estimator, X, y, cv=cv, scoring=recall_1_scorer)
    print("Promedio : %0.2f%%" % (crossv.mean()*100))
    print("Desv. Estandar : %0.2f%%" % (crossv.std()*100))
    print("----------------------")
    print("----------------------")
    print("Clase 0 : Retencion\nClasificaciones Correctas")
    print("----------------------")
    crossv = cross_validation.cross_val_score(estimator, X, y, cv=cv, scoring=recall_0_scorer)
    print("Promedio : %0.2f%%" % (crossv.mean()*100))
    print("Desv. Estandar : %0.2f%%" % (crossv.std()*100))
    print("----------------------")
    print("\n")
    return None

## Input

In [17]:
r = pd.read_csv('/Users/felipelolas/PycharmProjects/FracasoEscolarChile/DatasetsProcesados/SIMCE/ALU/SIMCE_GEO_2013-2014.csv', header=0, sep='|', decimal='.')
cols = ['MRUN','COD_COM_ALU','NOM_COM_ALU','SIT_FIN_R','EDAD_ALU','CODINE11','LAT_MANZANA_ALU','LON_MANZANA_ALU','RIESGO_DESERCION_RBD','DIR_RBD','LAT_MANZANA_RBD','LON_MANZANA_RBD',
        'CONVIVENCIA_2M_RBD','CONVIVENCIA_4B_RBD','CONVIVENCIA_6B_RBD','AUTOESTIMA_MOTIVACION_2M_RBD','AUTOESTIMA_MOTIVACION_4B_RBD','AUTOESTIMA_MOTIVACION_6B_RBD',
        'AUTOESTIMA_MOTIVACION_8B_RBD','PARTICIPACION_2M_RBD','PARTICIPACION_4B_RBD','PARTICIPACION_6B_RBD','PARTICIPACION_8B_RBD','IVE_MEDIA_RBD','IVE_BASICA_RBD',
        'PSU_PROM_2013_RBD','CPAD_DISP','DGV_RBD','NOM_RBD','NOM_COM_RBD','COD_COM_RBD','RBD','PROM_GRAL','ASISTENCIA','LET_CUR','CLASIFICACION_SEP_RBD']
data = r.drop(cols,1)
data = data.drop(data.columns[[0,21]],1)

## Preprocessing

In [18]:
#data = data.dropna()
#cdata['CLASIFICACION_SEP_RBD'].fillna('NO SEP', inplace=True)
nocat = ['CANT_TRASLADOS_ALU','CANT_DELITOS_COM_ALU','CANT_DELITOS_MANZANA_RBD','CANT_DOC_M_RBD','CANT_DOC_F_RBD','CANT_DOC_RBD','POB_FLOT_RBD','BECAS_DISP_RBD','MAT_TOTAL_RBD','VACANTES_CUR_IN_RBD','PROM_ALU_CUR_RBD','CANT_DELITOS_COM_RBD','EDU_P','EDU_M', 
         'ING_HOGAR','CANT_CURSOS_RBD','PAGO_MATRICULA_RBD','PAGO_MENSUAL_RBD']
output_data = data[['ABANDONA_ALU','DESERTA_ALU','ABANDONA_2014_ALU']]
data = data.drop(['ABANDONA_ALU','DESERTA_ALU','ABANDONA_2014_ALU'],1)
float_data = data.loc[:, data.dtypes == float]
object_data = data.loc[:, data.dtypes == object] # Se convierten en binarias desde texto
categorical_data = data.loc[:, data.dtypes == int].drop(nocat,1) # se convierten en binarias desde numeros sin escala
categorical_data = categorical_data[['GEN_ALU','ORI_RELIGIOSA_RBD']]
integer_data = data.loc[:, data.dtypes == int][nocat]

In [19]:
float_data_mapper = DataFrameMapper([(float_data.columns.values,[preprocessing.Imputer(missing_values=np.nan),preprocessing.MinMaxScaler((0,1))])])
integer_data_mapper = DataFrameMapper([(integer_data.columns.values,[preprocessing.Imputer(missing_values=np.nan),preprocessing.MinMaxScaler((0,1))])])
categorical_data_mapper = DataFrameMapper([(categorical_data.columns.values,preprocessing.Imputer(missing_values=-999))])
print("Imputando : 25%...")
float_data_arr = float_data_mapper.fit_transform(float_data) #Atributos Floats imputados y escalados
float_data = pd.DataFrame(float_data_arr,columns=float_data.columns)
print("Imputando : 50%...")
integer_data_arr = integer_data_mapper.fit_transform(integer_data) #Atributos Integer imputados y escalados
integer_data = pd.DataFrame(integer_data_arr,columns=integer_data.columns)
print("Imputando : 100%...")

print("Vectorizando : 0%...")
categorical_data_arr = categorical_data_mapper.fit_transform(categorical_data) #Atributos Categoricos numericos imputados
categorical_data = pd.DataFrame(categorical_data_arr,columns=categorical_data.columns)
## Vectorizar las categorias de texto
object_data_vectorizer = feature_extraction.DictVectorizer(sparse=False)
object_data_prep = object_data_vectorizer.fit_transform(object_data.T.to_dict().values())
object_data_prep_df = pd.DataFrame(object_data_prep, columns=object_data_vectorizer.get_feature_names())
print("Vectorizando : 50%...")
## Vectorizar las categorias numericas
categorical_data_vectorizer = feature_extraction.DictVectorizer(sparse=False)
categorical_data_prep = categorical_data_vectorizer.fit_transform(categorical_data.applymap(str).T.to_dict().values())
categorical_data_prep_df = pd.DataFrame(categorical_data_prep,columns=categorical_data_vectorizer.get_feature_names())
print("Vectorizando : 100%")
## Ahora escalamos los vectores binarios
#object_data_mapper_bin = DataFrameMapper([(object_data_prep_df.columns.values,preprocessing.MinMaxScaler((-1,1)))])
#categorical_data_mapper_bin = DataFrameMapper([(categorical_data_prep_df.columns.values,preprocessing.MinMaxScaler((-1,1)))])

#object_data_prep_arr_bin = object_data_mapper_bin.fit_transform(object_data_prep_df)

#categorical_data_prep_arr_bin = categorical_data_mapper_bin.fit_transform(categorical_data_prep_df)

#object_data_prep_df = pd.DataFrame(object_data_prep_arr_bin,columns=object_data_prep_df.columns)

#categorical_data_prep_df = pd.DataFrame(categorical_data_prep_arr_bin,columns=categorical_data_prep_df.columns)


Imputando : 25%...
Imputando : 50%...
Imputando : 100%...
Vectorizando : 0%...
Vectorizando : 50%...
Vectorizando : 100%


## Definicion INPUT y OUTPUT del Modelo

In [22]:
input_data = pd.concat([float_data, integer_data, categorical_data_prep_df, object_data_prep_df], axis=1, join='inner')

X = np.array(input_data)
y = np.array(output_data['DESERTA_ALU'])
t_names = ['Alumno no desertor(0)','Alumno desertor(1)']
##Scorers
recall_1_scorer = make_scorer(recall_score, pos_label=1,average='binary')
recall_0_scorer = make_scorer(recall_score, pos_label=0,average='binary')
roc_curve_scorer = make_scorer(roc_curve,pos_label=1)


## Hyperparameter Optimization

In [25]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = {
    'loss': ['log', 'hinge', 'squared_hinge', 'perceptron', 'huber', 'modified_huber', 'epsilon_insensitive'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001],
    'class_weight':['auto'],
    'n_jobs' :[-1]
}
scores = ['recall', 'recall_macro', 'recall_weighted']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SGDClassifier(), tuned_parameters, cv=10,
                       scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    #for params, mean_score, scores in clf.grid_scores_:
    #    print("%0.3f (+/-%0.03f) for %r"
    #          % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for recall
()
Best parameters set found on development set:
()
{'penalty': 'l1', 'alpha': 1e-06, 'loss': 'log', 'n_jobs': -1, 'class_weight': 'auto'}
()
Grid scores on development set:
()
()
Detailed classification report:
()
The model is trained on the full development set.
The scores are computed on the full evaluation set.
()
             precision    recall  f1-score   support

          0       1.00      0.82      0.90     67925
          1       0.10      0.88      0.19      1623

avg / total       0.98      0.82      0.88     69548

()
# Tuning hyper-parameters for recall_macro
()
Best parameters set found on development set:
()
{'penalty': 'l1', 'alpha': 0.0001, 'loss': 'log', 'n_jobs': -1, 'class_weight': 'auto'}
()
Grid scores on development set:
()
()
Detailed classification report:
()
The model is trained on the full development set.
The scores are computed on the full evaluation set.
()
             precision    recall  f1-score   support

       